# Solution to Exercise 6: T-type Ca²⁺-channels

The notebook-code below produces 6 figures.

## 1) Soma response
Figure 1 gives the solution to part 1 of the exercise. 

__a)__ With a synaptic weight $w$ = 0.01, the synaptic input triggers a burst in the control model (black curve). 

__b)__ When Na⁺ channels are blocked, the Ca²⁺-spike that the APs in the burst "rode upon" is revealed (red curve). When also T-channels are blocked, the neuron responds passively to the input (blue curve). 

__c)__ We note that before the synapse was activated (at 60 ms), the blue curve is slightly below the red and black curves, i.e. the cell has a lower resting potential in the case when T-channels are blocked. The explanation is that a small fraction of T-channels are open at the resting potential, and give rise to a small inward Ca²⁺ current which depolarizes the cell.

## 2) Dendritic signalling
The solution to part 2 of the exercise is found in Figures 2-4.

__a)__ Fig. 2 shows that the somatic APs only give rise to very small fuctuations in the distal dendrites. However,
the Ca²⁺-spikes are quite well preserved throughout the dendrites, and have roughly the same shape and amplitude in the dendritic endpoints as in the soma. In fact, the response in the distal dendrite is almost unaffected by blocking the Na⁺-channels (Fig. 3). There may be two overlapping explanations to this. (1) Dendrites act as low pass filters. Brief signals (action potentials) do not propagate as successfully as more enduring signals (Ca²⁺-spikes). (2) Ca²⁺-spikes do not just travel passively from the soma and out in the dendrites, but are
also (partly) generated locally in the dendrites by T-channels that are present there.

__b)__ Fig. 4 shows the response to a synaptic input in the distal dendrite. The local response at the synaptic site is large and peaks at 0 mV (bottom panel). The response in the soma (top panel) is much smaller, and the membrane potential in the soma only increases with about 1 mV. 

__c)__ Signal propagation is evidently more effective in the outward direction. This is generally the case due to the decreasing electronic load (going from the large soma towards gradually thinner structures). When we have active conductances in the dendrites, the answer to this question is less trivial. 

## 3) How does the distribution of T-channels affect the neural response properties?
The solution to part 3 of the exercise is found in Figures 5-6.

__a)__ The functions "active_conductances5" and "active_conductances6" show how a linearly increasing T-channel density was implemented in the model.

__b)__ 
* When changing from the uniform (Fig. 2) to a distance dependent (Fig. 5) distribution of T-channels, the somatic response to the somatic synapse was reduced (by 1 AP). When changing distributions, we essentially moved
T-channels away from the soma-region, thus increasing the average distance between the T-channels and the soma (site of the current injection). A reduction in the local somatic response was what thus expected. We also note that the onset of AP firing was later with a distance dependent T-distribution. This was because the burst depended on T-channels far away from the soma, i.e., the signal needed time to travel to the distal dendrites to activate the T-channels there. 
* Even if the synapse was in the soma and the soma-response was reduced, the distance dependent T-distribution gave a higher response in the distal dendrites compared to the uniform distribution (compare lower panels in Fig. 2 and Fig. 5). The explanation is the high local T-channel density in the distal dendrites. The dendritic response is not solely due to outward propagation of the somatic response, but is partly generated locally by the activation of local T-channels in distal dendrites.
* When the synapse was in the distal dendrite, the distance dependent T-distribution (Fig. 6) gave a larger response than the uniform distribution (Fig. 4) both at the synaptic site (not surprising) and in the soma (maybe surprising).

In summary, we can say that the uniform distribution facilitated the somatic response to somatic current injections, while the distance dependent distribution facilitated the conveyance of signals in both directions between the soma and distal dendrites.

In [ ]:
%matplotlib inline
import os
from os.path import join

retval = os.getcwd()
print("Current working directory %s" % retval)
os.chdir("halnes/")
!nrnivmodl
os.chdir(retval)

import numpy as np
import pylab as plt
import scipy.stats
import neuron
from neuron import h
import LFPy

np.random.seed(1234)


In [ ]:
def active_conductances1(**kwarg):
    rall = 113 # axial resistance
    cap = 1.1 # membrane capacitance
    Rm = 45000.0 # membrane resistance
    Epas = -70.6
    celsius = 36

    ## INSERT ION CHANNELS:
    for sec in h.allsec():
        sec.insert("pas")
        sec.e_pas = Epas
        sec.g_pas = 1/Rm
        sec.Ra = rall
        sec.cm = cap
        sec.insert("Cad")
        sec.insert("it2")
        sec.insert("hh2")
        sec.ena = 50 # Reversal potential for sodium
        sec.ek = -90 # Reversal potential for potassium

     ##################################################################
    # Channel densities
    gna = 0.18 # S/cm2
    gkdr = 0.4
    gcat = 1.e-4

    h.celsius = celsius
    for sec in h.soma:
        h.distance()

    for sec in h.allsec():
        sec.v = Epas
        sec.e_pas = Epas
        sec.insert("pas")
        sec.e_pas = Epas
        sec.g_pas = 1/Rm
        sec.Ra = rall
        sec.cm = cap
        sec.gnabar_hh2 = 0
        sec.gkbar_hh2 = 0
        for seg in sec:
            seg.gcabar_it2 = gcat
    for sec in h.soma:
        sec.gnabar_hh2 = gna
        sec.gkbar_hh2 = gkdr
        seg.gcabar_it2 = gcat

def active_conductances2(**kwarg):
    rall = 113 # axial resistance
    cap = 1.1 # membrane capacitance
    Rm = 45000.0 # membrane resistance
    Epas = -70.6
    celsius = 36

    ## INSERT ION CHANNELS:
    for sec in h.allsec():
        sec.insert("pas")
        sec.e_pas = Epas
        sec.g_pas = 1/Rm
        sec.Ra = rall
        sec.cm = cap
        sec.insert("Cad")
        sec.insert("it2")
        sec.insert("hh2")
        sec.ena = 50 # Reversal potential for sodium
        sec.ek = -90 # Reversal potential for potassium

     ##################################################################
    # Channel densities
    gna = 0 # S/cm2
    gkdr = 0.4
    gcat = 1.e-4

    h.celsius = celsius
    for sec in h.soma:
        h.distance()

    for sec in h.allsec():
        sec.v = Epas
        sec.e_pas = Epas
        sec.insert("pas")
        sec.e_pas = Epas
        sec.g_pas = 1/Rm
        sec.Ra = rall
        sec.cm = cap
        sec.gnabar_hh2 = 0
        sec.gkbar_hh2 = 0
        for seg in sec:
            seg.gcabar_it2 = gcat
    for sec in h.soma:
        sec.gnabar_hh2 = gna
        sec.gkbar_hh2 = gkdr
        seg.gcabar_it2 = gcat
        
def active_conductances3(**kwarg):
    rall = 113 # axial resistance
    cap = 1.1 # membrane capacitance
    Rm = 45000.0 # membrane resistance
    Epas = -70.6
    celsius = 36

    ## INSERT ION CHANNELS:
    for sec in h.allsec():
        sec.insert("pas")
        sec.e_pas = Epas
        sec.g_pas = 1/Rm
        sec.Ra = rall
        sec.cm = cap
        sec.insert("Cad")
        sec.insert("it2")
        sec.insert("hh2")
        sec.ena = 50 # Reversal potential for sodium
        sec.ek = -90 # Reversal potential for potassium

     ##################################################################
    # Channel densities
    gna = 0.18 # S/cm2
    gkdr = 0.4
    gcat = 1.e-4

    h.celsius = celsius
    for sec in h.soma:
        h.distance()

    for sec in h.allsec():
        sec.v = Epas
        sec.e_pas = Epas
        sec.insert("pas")
        sec.e_pas = Epas
        sec.g_pas = 1/Rm
        sec.Ra = rall
        sec.cm = cap
        sec.gnabar_hh2 = 0
        sec.gkbar_hh2 = 0
        for seg in sec:
            seg.gcabar_it2 = gcat
    for sec in h.soma:
        sec.gnabar_hh2 = gna
        sec.gkbar_hh2 = gkdr
        seg.gcabar_it2 = gcat
        
def active_conductances4(**kwarg):
    rall = 113 # axial resistance
    cap = 1.1 # membrane capacitance
    Rm = 45000.0 # membrane resistance
    Epas = -70.6
    celsius = 36

    ## INSERT ION CHANNELS:
    for sec in h.allsec():
        sec.insert("pas")
        sec.e_pas = Epas
        sec.g_pas = 1/Rm
        sec.Ra = rall
        sec.cm = cap
        sec.insert("Cad")
        sec.insert("it2")
        sec.insert("hh2")
        sec.ena = 50 # Reversal potential for sodium
        sec.ek = -90 # Reversal potential for potassium

     ##################################################################
    # Channel densities
    gna = 0.18 # S/cm2
    gkdr = 0.4
    gcat = 1.e-4

    h.celsius = celsius
    for sec in h.soma:
        h.distance()

    for sec in h.allsec():
        sec.v = Epas
        sec.e_pas = Epas
        sec.insert("pas")
        sec.e_pas = Epas
        sec.g_pas = 1/Rm
        sec.Ra = rall
        sec.cm = cap
        sec.gnabar_hh2 = 0
        sec.gkbar_hh2 = 0
        for seg in sec:
            seg.gcabar_it2 = gcat*0.1054 *(1 + 0.04 * h.distance(seg.x))
    for sec in h.soma:
        sec.gnabar_hh2 = gna
        sec.gkbar_hh2 = gkdr
        seg.gcabar_it2 = gcat
        
def active_conductances5(**kwarg):
    rall = 113 # axial resistance
    cap = 1.1 # membrane capacitance
    Rm = 45000.0 # membrane resistance
    Epas = -70.6
    celsius = 36

    ## INSERT ION CHANNELS:
    for sec in h.allsec():
        sec.insert("pas")
        sec.e_pas = Epas
        sec.g_pas = 1/Rm
        sec.Ra = rall
        sec.cm = cap
        sec.insert("Cad")
        sec.insert("it2")
        sec.insert("hh2")
        sec.ena = 50 # Reversal potential for sodium
        sec.ek = -90 # Reversal potential for potassium

     ##################################################################
    # Channel densities
    gna = 0.18 # S/cm2
    gkdr = 0.4
    gcat = 1.e-4

    h.celsius = celsius
    for sec in h.soma:
        h.distance()

    for sec in h.allsec():
        sec.v = Epas
        sec.e_pas = Epas
        sec.insert("pas")
        sec.e_pas = Epas
        sec.g_pas = 1/Rm
        sec.Ra = rall
        sec.cm = cap
        sec.gnabar_hh2 = 0
        sec.gkbar_hh2 = 0
        for seg in sec:
            seg.gcabar_it2 = gcat*0.1054 *(1 + 0.04 * h.distance(seg.x))
    for sec in h.soma:
        sec.gnabar_hh2 = gna
        sec.gkbar_hh2 = gkdr
        seg.gcabar_it2 = gcat

def active_conductances6(**kwarg):
    rall = 113 # axial resistance
    cap = 1.1 # membrane capacitance
    Rm = 45000.0 # membrane resistance
    Epas = -70.6
    celsius = 36

    ## INSERT ION CHANNELS:
    for sec in h.allsec():
        sec.insert("pas")
        sec.e_pas = Epas
        sec.g_pas = 1/Rm
        sec.Ra = rall
        sec.cm = cap
        sec.insert("Cad")
        sec.insert("it2")
        sec.insert("hh2")
        sec.ena = 50 # Reversal potential for sodium
        sec.ek = -90 # Reversal potential for potassium

     ##################################################################
    # Channel densities
    gna = 0 # S/cm2
    gkdr = 0.4
    gcat = 0

    h.celsius = celsius
    for sec in h.soma:
        h.distance()

    for sec in h.allsec():
        sec.v = Epas
        sec.e_pas = Epas
        sec.insert("pas")
        sec.e_pas = Epas
        sec.g_pas = 1/Rm
        sec.Ra = rall
        sec.cm = cap
        sec.gnabar_hh2 = 0
        sec.gkbar_hh2 = 0
        for seg in sec:
            seg.gcabar_it2 = gcat*0.1054 *(1 + 0.04 * h.distance(seg.x))
    for sec in h.soma:
        sec.gnabar_hh2 = gna
        sec.gkbar_hh2 = gkdr
        seg.gcabar_it2 = gcat
        
def return_interneuron(end_T, dt, start_T, i, v_init=-75., ):
    neuron.load_mechanisms(join("halnes"))
    if i == 1:
        cell = LFPy.Cell(morphology=join("halnes", "Morf_default.hoc"),
                     dt=dt,
                     tstart=start_T,
                     tstop=end_T,
                     nsegs_method='lambda_f',
                     lambda_f=100,
                     custom_fun=[active_conductances1],
                     custom_fun_args=[{}],
                     )
    if i == 2:
        cell = LFPy.Cell(morphology=join("halnes", "Morf_default.hoc"),
                     dt=dt,
                     tstart=start_T,
                     tstop=end_T,
                     nsegs_method='lambda_f',
                     lambda_f=100,
                     custom_fun=[active_conductances2],
                     custom_fun_args=[{}],
                     )
    if i == 3:
        cell = LFPy.Cell(morphology=join("halnes", "Morf_default.hoc"),
                     dt=dt,
                     tstart=start_T,
                     tstop=end_T,
                     nsegs_method='lambda_f',
                     lambda_f=100,
                     custom_fun=[active_conductances3],
                     custom_fun_args=[{}],
                     )
    if i == 4:
        cell = LFPy.Cell(morphology=join("halnes", "Morf_default.hoc"),
                     dt=dt,
                     tstart=start_T,
                     tstop=end_T,
                     nsegs_method='lambda_f',
                     lambda_f=100,
                     custom_fun=[active_conductances4],
                     custom_fun_args=[{}],
                     )
    if i == 5:
        cell = LFPy.Cell(morphology=join("halnes", "Morf_default.hoc"),
                     dt=dt,
                     tstart=start_T,
                     tstop=end_T,
                     nsegs_method='lambda_f',
                     lambda_f=100,
                     custom_fun=[active_conductances5],
                     custom_fun_args=[{}],
                     )
    if i == 6:
        cell = LFPy.Cell(morphology=join("halnes", "Morf_default.hoc"),
                     dt=dt,
                     tstart=start_T,
                     tstop=end_T,
                     nsegs_method='lambda_f',
                     lambda_f=100,
                     custom_fun=[active_conductances6],
                     custom_fun_args=[{}],
                     )
        
    cell.simulate(rec_imem=True, rec_vmem=True)
    return cell

In [ ]:
def insert_synapses(cell, weight, pos):
    '''Insert synapses'''
    
    synapse_params = {
        'e' : 0,                   #reversal potential
        'syntype' : 'Exp2Syn',      #synapse type
        'tau1' : 1,                 #syn time constant
        'tau2' : 3,                 #syn time constant
        'weight' : weight,           #syn weight
        'record_current' : True    #syn current gets recorded
    }
    
    idx = np.array([pos])
    args = dict(n=1, tstart=cell.tstart, tstop=cell.tstop,
                  distribution=scipy.stats.gamma,
                  rvs_args=dict(a=0.5, loc=0., scale=40)
                  )
    #Insert synapses in an iterative fashion
    for i in idx:
        synapse_params['idx'] = int(i)
        spiketimes = np.array([60.0])
        # Create synapse(s) and setting times using the Synapse class in LFPy
        s = LFPy.Synapse(cell, **synapse_params)
        s.set_spike_times(spiketimes)
    return s

In [ ]:
def plot_signal(cell, synapse, synpos, mytitle):
    """
    Plots the somatic membrane potential and somatic transmembrane currents (left), 
    as well as the cell morphology (center), and the extracellular potential (right).
    
    Parameters:
    ===========
    cell : LFPy.Cell object that contains the cell model. This function assumes that the cell simulation is finished
    electrode: Electrode object from the LFPy class RecExtElectrode 
    """
#    plt.close("all")
    cell_plot_idx = 0
    fig = plt.figure(figsize=[12, 9])
    fig.subplots_adjust(hspace=0.5)  # Adjusts the vertical distance between panels.
    ax_cell = fig.add_subplot(122, aspect='equal')
    ax_cell.axis('off')
    ipos = np.array([0, 65, 110, 128]) # Found by trial and error   
    
    [ax_cell.plot([cell.xstart[idx], cell.xend[idx]], 
              [cell.ystart[idx], cell.yend[idx]], c='k') for idx in range(cell.totnsegs)]
    ax_cell.plot(cell.xmid[ipos[0]], cell.ymid[ipos[0]], '*', c='k', ms=14)
    ax_cell.plot(cell.xmid[ipos[1]], cell.ymid[ipos[1]], '*', c='b', ms=14)
    ax_cell.plot(cell.xmid[ipos[2]], cell.ymid[ipos[2]], '*', c='g', ms=14)
    ax_cell.plot(cell.xmid[ipos[3]], cell.ymid[ipos[3]], '*', c='r', ms=14)
    if synpos == ipos[0]:
        syncol = 'k'
    elif synpos == ipos[1]:
        syncol = 'b'
    elif synpos == ipos[2]:
        syncol = 'g'
    elif synpos == ipos[3]:
        syncol = 'r'
    else:
        syncol = 'm'

    el, = ax_cell.plot(cell.xmid[synpos], cell.ymid[synpos], 'd', c=syncol, ms = 16)    
    ax_cell.legend([el], ["Synapse"], frameon=False, loc="lower center")

    ax_v = fig.add_subplot(521, ylabel='mV')
    ax_v.set_title(mytitle)
    ax_v.plot(cell.tvec, cell.somav, color='k', lw=2)
    ax_v = fig.add_subplot(523, ylabel='mV')
    ax_v.plot(cell.tvec, cell.vmem[ipos[1]], color='b', lw=2)
    ax_v = fig.add_subplot(525, ylabel='mV')
    ax_v.plot(cell.tvec, cell.vmem[ipos[2]], color='g', lw=2)
    ax_v = fig.add_subplot(527, xlabel='Time [ms]', ylabel='mV')
    ax_v.plot(cell.tvec, cell.vmem[ipos[3]], color='r', lw=2)
    ax_v = fig.add_subplot(529, xlabel='Time [ms]', ylabel='nA')    
    ax_v.plot(cell.tvec, synapse.i, color= syncol, lw=2)

In [ ]:
cell_name = "halnes"
weight = 0.01
somapos = 0
distdendpos = 128
closedendpos = 65
fardendpos = 110
        
cell_folder = join(cell_name)
end_T = 200
dt = 2**-5
start_T = -100 
jointfig = plt.figure(figsize=[6, 4])
jax_v = jointfig.add_subplot(111, xlabel='ms', ylabel='mV')
jax_v.set_title('Fig. 1: soma response. Uniform T-dist.')
    
for i in range(1, 7):
    if i == 1:
        titlestring = 'Fig. 2: Uniform T-dist. Somatic input'
        synpos = somapos
    if i == 2:
        titlestring = 'Fig. 3: Na blocked. Somatic input'
        synpos = somapos
    if i == 3:
        titlestring = 'Fig. 4: Uniform T-dist. Distal input'
        synpos = distdendpos
    if i == 4:
        titlestring = 'Fig. 5: Linear T-dist. Somatic input'
        synpos = somapos
    if i == 5:
        titlestring = 'Fig. 6: Linear T-dist. Distal input'
        synpos = distdendpos
    if i == 6:
        #Na & T blocked. Somatic input. No separate figure for this.
        synpos = somapos
        
    cell = return_interneuron(end_T, dt, start_T, i)
    synapse = insert_synapses(cell, weight=weight, pos = synpos)
    cell.simulate(rec_imem=False, rec_vmem=True, synpos = synpos)

    if i == 1:
        jax_v.plot(cell.tvec, cell.somav, label='control', color='k', lw=2)
    if i == 2:
        jax_v.plot(cell.tvec, cell.somav, label='Na blocked', color='r', lw=2)
    if i == 6:
        jax_v.plot(cell.tvec, cell.somav, label='Na & Ca blocked', color='b', lw=2)
        jax_v.legend()
        
    if not i == 6:
        plot_signal(cell, synapse, synpos, titlestring)